In [8]:
import requests
import json
import os
from dotenv import load_dotenv

In [9]:
load_dotenv()  # take environment variables

True

In [10]:
MTA_API_KEY = os.getenv("MTA_API_KEY")

In [11]:
# getting data given a route id
def get_bus_route(route_id):
    url = f"https://bustime.mta.info/api/where/stops-for-route/MTA%20NYCT_{route_id}.json?key={MTA_API_KEY}&includePolylines=false&version=2"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [12]:
route_id = "B63" # Example route
bus_data = get_bus_route(route_id)
print(type(bus_data))



<class 'dict'>


In [13]:
def get_stop_groups(bus_data):
    if not bus_data['data']['entry']['stopGroupings'][0]['ordered']:
        print(f"{bus_data['routeId']} not ordered")
        return
    route_lists = {}
    stopGroups = bus_data['data']['entry']['stopGroupings'][0]['stopGroups']
    
    for group in stopGroups:

        route_lists[group['name']['name']] = group['stopIds']
    
    return route_lists

    

In [14]:
test_route_lists = get_stop_groups(bus_data)

In [20]:
json_path='./graph_data/bus_info_dict.json'
with open(json_path) as json_file:
    bus_info = json.load(json_file)

In [21]:
print(bus_info['MTA_901601'])

{'name': 'Brooklyn Bridge Park /PIER 6', 'routes': ['B63'], 'direction': 'W', 'link': 'https://bustime.mta.info/m/index?q=901601', 'id': 'MTA_901601', 'latitude': 40.693078, 'longitude': -74.000877, 'osmid': 1567286111}


In [23]:
json_path='./graph_data/all_routes.json'
with open(json_path) as json_file:
    all_routes = json.load(json_file)

In [25]:
print(len(all_routes))

357
